In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('savano-mal-dataset/anime.csv', delimiter='\t')

In [ ]:
# List of phrases like sequels and prequels to check for so that we can remove the rows containing these phrases and just keep the original show
season_phrases_to_remove = ["2nd season", "3rd season", "4th season", "5th season", "6th season", 
                     "second season", "third season", "fourth season", "fifth season", "sixth season",
                     "second continuation", "third continuation", "fourth continuation", "fifth continuation", "sixth continuation",
                     "second series", "third series", "fourth series", "fifth series", "sixth series",
                     "second show", "third show", "fourth show", "fifth show", "sixth show"
                     "second animated", "third animated", "fourth animated", "fifth animated", "sixth animated",
                     "2nd animated", "3rd animated", "4th animated", "5th animated", "6th animated",
                     "sequel", "prequel", "adaptation", "continuation", "bundled", "aired", "bonus episode", 
                     "dvd", "based on", "series", "episode", "episodes", "ova", "specials", ":", "movie"]

# List of inappropriate phrases to check for so that we can remove the rows containing these phrases
inappropriate_phrases_to_remove = ["erotica", "hentai"]

# Create a boolean masks for rows to filter them out later
synopsis_season_mask = df['synopsis'].str.contains('|'.join(season_phrases_to_remove), case=False)
title_season_mask = df['title'].str.contains('|'.join(season_phrases_to_remove), case=False)
inappropriate_phrases_mask = df['genres'].str.contains('|'.join(inappropriate_phrases_to_remove), case=False)
combined_phrases_mask = synopsis_season_mask | title_season_mask | inappropriate_phrases_mask

# Use boolean indexing to keep only rows where the mask is False (phrases are not present)
cleaned_phrases_df = df[~combined_phrases_mask]

In [ ]:
# Create a regex pattern to match a range of characters outside ASCII using Unicode character codes
pattern = "[\u0080-\uFFFF]"  # Matches characters outside the ASCII range

# Create a boolean masks for rows to filter them out later
synopsis_ascii_mask = cleaned_phrases_df['synopsis'].str.contains(pattern, regex=True)
title_ascii_mask = cleaned_phrases_df['title'].str.contains(pattern, regex=True)
combined_ascii_mask = synopsis_ascii_mask | title_ascii_mask

# Use boolean indexing to keep only rows where the mask is False (characters outside ASCII range are not present)
cleaned_ascii_df = cleaned_phrases_df[~combined_ascii_mask]

In [ ]:
# Function to count words in a cell
def count_words(cell_value):
    if isinstance(cell_value, str):
        words = cell_value.split()
        return len(words)
    else:
        return 0  # Return 0 for non-string values

# Apply the function to each cell in the 'synopsis' column and create a boolean mask
word_counts = cleaned_ascii_df['synopsis'].apply(count_words)
mask = word_counts > 22 

# Use boolean indexing to keep only rows where the word count is > 22
word_num_df = cleaned_ascii_df[mask]

In [ ]:
# Make a copy of original df with relevant columns for value manipulation
cleaned_df = word_num_df[['title', 'synopsis', 'genres', 'start_date', 'status', 'popularity_rank']].copy()
# Remove the "Source: XYZ" text from 'synopsis' column
cleaned_df['synopsis'] = cleaned_df['synopsis'].str.replace(r'.{1}Source:.*', '', regex=True)

In [ ]:
# Convert 'start_date' column to "start_year" and remove null values
cleaned_df['start_date'] = pd.to_datetime(cleaned_df['start_date'])
cleaned_df['start_date'] = cleaned_df['start_date'].dt.year
cleaned_df.rename(columns={'start_date': 'start_year'}, inplace=True)
cleaned_df = cleaned_df[~cleaned_df.start_year.isnull()]

In [ ]:
cleaned_df.to_csv('cleanedQuizData.csv', index=False)